## Install dependencies

In [ ]:
%%capture
!sudo apt install git-lfs

In [ ]:
!pip install "transformers==4.26.0" "datasets==2.9.0" "accelerate==0.16.0" "evaluate==0.4.0"
!pip install "deepspeed==0.8.0"
!pip install rouge-score nltk py7zr tensorboard


In [1]:
import deepspeed
deepspeed.ops.op_builder.CPUAdamBuilder().load()

/opt/conda/envs/dev/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Installed CUDA version 11.7 does not match the version torch was compiled with 11.2 but since the APIs are compatible, accepting this combination


Using /home/ubuntu/.cache/torch_extensions/py39_cu112 as PyTorch extensions root...
Creating extension directory /home/ubuntu/.cache/torch_extensions/py39_cu112/cpu_adam...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py39_cu112/cpu_adam/build.ninja...
Building extension module cpu_adam...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] /opt/conda/envs/dev/bin/nvcc  -DTORCH_EXTENSION_NAME=cpu_adam -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1014\" -I/opt/conda/envs/dev/lib/python3.9/site-packages/deepspeed/ops/csrc/includes -I/opt/conda/envs/dev/include -isystem /opt/conda/envs/dev/lib/python3.9/site-packages/torch/include -isystem /opt/conda/envs/dev/lib/python3.9/site-packages/torch/include/torch/csrc/api/include -isystem /opt/conda/envs/dev/lib/python3.9/site-packages/torch/include/TH -isystem /opt/conda/envs/dev/lib/python3.9/site-packages/torch/include/THC -isystem /opt/conda/envs/dev/include -isystem /opt/conda/envs/dev/include/python3.9 -D_GLIBCXX_USE_CXX11_ABI=1 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencode=arch=compute_70,code=compute_70 -gencode=arch=compute_70,code=sm_70 --compiler-options '-fPIC' -O3 --use

RuntimeError: Error building extension 'cpu_adam'

In [1]:
conda search -c pytorch pytorch=1.13.1=py3.8_cuda11.7*


Loading channels: done
# Name                       Version           Build  Channel             
pytorch                       1.13.1 py3.8_cuda11.7_cudnn8.5.0_0  pytorch             

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch

torch.version.cuda

'11.2'

In [3]:
!ds_report

--------------------------------------------------
DeepSpeed C++/CUDA extension op report
--------------------------------------------------
NOTE: Ops not installed will be just-in-time (JIT) compiled at
      runtime if needed. Op compatibility means that your system
      meet the required dependencies to JIT install the op.
--------------------------------------------------
JIT compiled ops requires ninja
ninja .................. [OKAY]
--------------------------------------------------
op name ................ installed .. compatible
--------------------------------------------------
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
async_io ............... [NO] ....... [NO]
cpu_adagrad ............ [NO] ....... [OKAY]
cpu_a

# Process dataset

In [4]:
# experiment config
model_id = "google/flan-t5-base"
repository_id = "flan-t5-base-cnn"

# Dataset 
dataset_id = "cnn_dailymail"
dataset_config = "3.0.0"
save_dataset_path = "data"
text_column = "article"
summary_column = "highlights"
prompt_start = "Summarize the following news article:\n"
generation_start = "\nSummary:\n"
prompt_template = f"{prompt_start}{{input}}{generation_start}"

In [15]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np 


# Load dataset from the hub
dataset = load_dataset(dataset_id,name=dataset_config)
# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Found cached dataset cnn_dailymail (/home/ubuntu/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)


  0%|          | 0/3 [00:00<?, ?it/s]

Train dataset size: 287113
Test dataset size: 11490


In [16]:
prompt_lenght = len(tokenizer(prompt_template.format(input=""))["input_ids"])
max_sample_length = tokenizer.model_max_length - prompt_lenght
print(f"Prompt lenght: {prompt_lenght}")
print(f"Max input lenght: {max_sample_length}")

Prompt lenght: 12
Max input lenght: 500


In [24]:
from datasets import concatenate_datasets
import numpy as np

# The maximum total input sequence length after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x[text_column], truncation=True), batched=True, remove_columns=[text_column, summary_column])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
max_source_length = min(max_source_length, max_sample_length)
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x[summary_column], truncation=True), batched=True, remove_columns=[text_column, summary_column])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# use 95th percentile as max target length
max_target_length = int(np.percentile(target_lenghts, 95))
print(f"Max target length: {max_target_length}")

  0%|          | 0/299 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de/cache-2243e7f93d3afe60.arrow


Max source length: 500
Max target length: 129


In [5]:
max_target_length=129

In [29]:
import os

def preprocess_function(sample,padding="max_length"):
    # created prompted input
    inputs = [prompt_template.format(input=item) for item in sample[text_column]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample[summary_column], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=list(dataset["train"].features))
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

tokenized_dataset["train"].save_to_disk(os.path.join(save_dataset_path,"train"))
tokenized_dataset["test"].save_to_disk(os.path.join(save_dataset_path,"eval"))

  0%|          | 0/288 [00:00<?, ?ba/s]

  0%|          | 0/14 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


Saving the dataset (0/3 shards):   0%|          | 0/287113 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/11490 [00:00<?, ? examples/s]

## Launch Training


In [3]:
import nltk
nltk.download("punkt", quiet=True)

True

In [41]:
!python scripts/run_seq2seq_deepspeed.py \
    --model_id $model_id \
    --dataset_path $save_dataset_path \
    --repository_id $repository_id \
    --epochs 3 \
    --per_device_train_batch_size 8 \
    --per_device_eval_batch_size 8 \
    --generation_max_length $max_target_length \
    --lr 5e-5 

  0%|                                   | 131/107670 [00:59<13:39:54,  2.19it/s]^C


In [6]:
!deepspeed --num_gpus=1 scripts/run_seq2seq_deepspeed.py \
    --model_id $model_id \
    --dataset_path $save_dataset_path \
    --epochs 3 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 16 \
    --generation_max_length $max_target_length \
    --lr 5e-5 \
    --deepspeed configs/ds_flan_t5_z3_config.json \
    --repository_id $repository_id \

[2023-02-03 15:35:15,007] [WARNING] [runner.py:186:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
[2023-02-03 15:35:15,022] [INFO] [runner.py:548:main] cmd = /opt/conda/envs/pytorch/bin/python3.9 -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None scripts/run_seq2seq_deepspeed.py --model_id google/flan-t5-base --dataset_path data --epochs 3 --per_device_train_batch_size 16 --per_device_eval_batch_size 16 --generation_max_length 129 --lr 5e-5 --deepspeed configs/ds_flan_t5_z3_config.json --repository_id flan-t5-base-cnn \
[2023-02-03 15:35:17,522] [INFO] [launch.py:142:main] WORLD INFO DICT: {'localhost': [0]}
[2023-02-03 15:35:17,523] [INFO] [launch.py:148:main] nnodes=1, num_local_procs=1, node_rank=0
[2023-02-03 15:35:17,523] [INFO] [launch.py:161:main] global_rank_mapping=defaultdict(<class 'list'>, {'localhost': [0]})
[2023-02-03 15:35:17,523] [I